In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

# --- YAPILANDIRMA VE SABİTLER ---
DATA_ROOT = '/kaggle/input/nfl-big-data-bowl-2026-prediction/'
SEED = 42

# Eğitim (Train) ve Test dosyalarının yollarını dinamik olarak bulma
def find_data_files(base_path, prefix):
    return glob.glob(os.path.join(base_path, f'train/{prefix}_*.csv'))

# ==============================================================================
# 1. YARDIMCI FONKSİYONLAR
# ==============================================================================

def normalize_coordinates(df):
    """Sadece 'play_direction' sütununa sahip DataFrame'ler için koordinatları normalize eder."""
    df_copy = df.copy()
    if 'play_direction' not in df_copy.columns:
        return df_copy
    
    left_plays = df_copy['play_direction'] == 'left'
    
    for col in ['x', 'ball_land_x']:
        if col in df_copy.columns:
            df_copy.loc[left_plays, col] = 120 - df_copy.loc[left_plays, col]
    for col in ['y', 'ball_land_y']:
        if col in df_copy.columns:
            df_copy.loc[left_plays, col] = 53.3 - df_copy.loc[left_plays, col]
    
    for col in ['dir', 'o']:
        if col in df_copy.columns:
            df_copy.loc[left_plays, col] = np.mod(df_copy.loc[left_plays, col] + 180, 360)
            
    return df_copy

def apply_feature_engineering(df):
    """Temel hareket vektörü özelliklerini oluşturur."""
    df_copy = df.copy()
    
    required_cols = ['x', 'y', 'ball_land_x', 'ball_land_y', 'num_frames_output', 's', 'dir']
    if any(col not in df_copy.columns for col in required_cols):
        # Bu hata, test_df yüklenirken yaşanabilir.
        print(f"UYARI: Özellik mühendisliği için gerekli bazı sütunlar eksik: {required_cols}")
        return df_copy 
    
    # 1. Toplanma noktasına uzaklık ve açısal özellikler
    df_copy['dist_to_land'] = np.sqrt(
        (df_copy['x'] - df_copy['ball_land_x'])**2 + 
        (df_copy['y'] - df_copy['ball_land_y'])**2
    )

    # 2. Çerçeve başına hedeflenen ortalama hareket vektörü
    df_copy['target_dx_per_frame'] = (df_copy['ball_land_x'] - df_copy['x']) / df_copy['num_frames_output']
    df_copy['target_dy_per_frame'] = (df_copy['ball_land_y'] - df_copy['y']) / df_copy['num_frames_output']
    df_copy['target_dist_per_frame'] = np.sqrt(
        df_copy['target_dx_per_frame']**2 + df_copy['target_dy_per_frame']**2
    )
    
    # 3. Oyuncu hızının bileşenleri
    df_copy['dx'] = df_copy['s'] * np.cos(np.deg2rad(df_copy['dir']))
    df_copy['dy'] = df_copy['s'] * np.sin(np.deg2rad(df_copy['dir']))

    # 4. Hız farkı
    df_copy['speed_diff_x'] = df_copy['target_dx_per_frame'] - df_copy['dx']
    df_copy['speed_diff_y'] = df_copy['target_dy_per_frame'] - df_copy['dy']
    
    return df_copy


# ==============================================================================
# 2. VERİ YÜKLEME VE EĞİTİM
# ==============================================================================

print("Veri yükleniyor ve modeller eğitiliyor...")

input_files = find_data_files(DATA_ROOT, 'input')
output_files = find_data_files(DATA_ROOT, 'output')

if not input_files:
    raise FileNotFoundError("Eğitim verisi dosyaları bulunamadı.")
    
train_input_df = pd.concat([pd.read_csv(f) for f in input_files])
train_output_df = pd.concat([pd.read_csv(f) for f in output_files])

# Birleştirme (Suffixes, sütun isimlerini değiştirir)
train_df = pd.merge(
    train_input_df, train_output_df, 
    on=['game_id', 'play_id', 'nfl_id', 'frame_id'], 
    suffixes=('_start', '_end'), 
    how='inner'
)
del train_input_df, train_output_df

# Hedef değişkenler (Deplasman)
train_df['target_dx'] = train_df['x_end'] - train_df['x_start']
train_df['target_dy'] = train_df['y_end'] - train_df['y_start']

# 🚨 HATA GİDERME KRİTİK ADIM: Feature Engineering için sütunları yeniden adlandır
rename_map = {
    'x_start': 'x', 'y_start': 'y', 's_start': 's', 'a_start': 'a', 
    'dir_start': 'dir', 'o_start': 'o', 
    'ball_land_x_start': 'ball_land_x', 'ball_land_y_start': 'ball_land_y',
    'num_frames_output_start': 'num_frames_output',
}
train_df.rename(columns=rename_map, inplace=True)

# Koordinat Normalizasyonu ve Özellik Mühendisliği
train_df = normalize_coordinates(train_df)
train_df = apply_feature_engineering(train_df)

# Özellik seçimi (Yeni ve orijinal özellikler bir arada)
FEATURES = [
    'x', 'y', 's', 'a', 'dir', 'o', 
    'ball_land_x', 'ball_land_y', 'num_frames_output',
    'dist_to_land', 'target_dx_per_frame', 'target_dy_per_frame', 
    'target_dist_per_frame', 'dx', 'dy', 'speed_diff_x', 'speed_diff_y'
]

# NaN değerleri doldurma (eğitim verisi için)
for col in FEATURES:
    if col in train_df.columns:
        train_df[col].fillna(train_df[col].median(), inplace=True)
    
X = train_df[FEATURES]
Y_DX = train_df['target_dx']
Y_DY = train_df['target_dy']

# Model Eğitimi (LightGBM)
LGBM_PARAMS = {
    'objective': 'regression',
    'metric': 'rmse',
    'n_estimators': 1500,
    'learning_rate': 0.03,
    'n_jobs': -1,
    'seed': SEED,
    'colsample_bytree': 0.7,
    'subsample': 0.7
}

lgbm_dx = LGBMRegressor(**LGBM_PARAMS)
lgbm_dy = LGBMRegressor(**LGBM_PARAMS)

lgbm_dx.fit(X, Y_DX)
lgbm_dy.fit(X, Y_DY)

print("✅ Modeller Başarıyla Eğitildi.")


# ==============================================================================
# 3. TAHMİN VE SUBMISSION DOSYASI OLUŞTURMA
# ==============================================================================

print("\n--- Tahminler Yapılıyor ve submission.csv Oluşturuluyor ---")

TEST_INPUT_PATH = os.path.join(DATA_ROOT, 'test_input.csv')
SAMPLE_SUB_PATH = os.path.join(DATA_ROOT, 'sample_submission.csv')

test_df = pd.read_csv(TEST_INPUT_PATH)
sample_submission_df = pd.read_csv(SAMPLE_SUB_PATH)

# Test verisini hazılama (test_input.csv zaten doğru sütun isimlerine sahiptir)
test_df_processed = normalize_coordinates(test_df.copy())
test_df_processed = apply_feature_engineering(test_df_processed)

# Test verisindeki NaN değerleri 0 ile doldurma (eğitim verisinin medyanı yok)
for col in FEATURES:
    if col in test_df_processed.columns:
        test_df_processed[col].fillna(0, inplace=True) 

X_test = test_df_processed[FEATURES]

# Tahmin yapma: Deplasman (displacement) tahmin ediliyor
pred_dx = lgbm_dx.predict(X_test)
pred_dy = lgbm_dy.predict(X_test)

# Orijinal koordinatları kullanarak son konumu hesaplama (normalize edilmiş alanda)
test_df_processed['x_pred_norm'] = test_df_processed['x'] + pred_dx
test_df_processed['y_pred_norm'] = test_df_processed['y'] + pred_dy

# Ters Normalizasyon ve Alan Sınırlandırması (CLIP)
left_plays = test_df_processed['play_direction'] == 'left'

# x
test_df_processed.loc[left_plays, 'x_final'] = 120 - test_df_processed.loc[left_plays, 'x_pred_norm']
test_df_processed.loc[~left_plays, 'x_final'] = test_df_processed.loc[~left_plays, 'x_pred_norm']

# y
test_df_processed.loc[left_plays, 'y_final'] = 53.3 - test_df_processed.loc[left_plays, 'y_pred_norm']
test_df_processed.loc[~left_plays, 'y_final'] = test_df_processed.loc[~left_plays, 'y_pred_norm']

# 🚨 KRİTİK: Koordinatları NFL saha sınırları içine zorla
test_df_processed['x_final'] = test_df_processed['x_final'].clip(0.0, 120.0)
test_df_processed['y_final'] = test_df_processed['y_final'].clip(0.0, 53.3)


# --- Submission Dosyasını Oluşturma (ID birleştirme) ---

# Birleşik 'id' sütununu oluştur
test_df_processed['id'] = (
    test_df_processed['game_id'].astype(str) + '_' + 
    test_df_processed['play_id'].astype(str) + '_' + 
    test_df_processed['nfl_id'].astype(str) + '_' + 
    test_df_processed['frame_id'].astype(str)
)

# Merge işlemi
final_submission_df = pd.merge(
    sample_submission_df[['id']], 
    test_df_processed[['id', 'x_final', 'y_final']],
    on='id', 
    how='left'
)

# Sütunları yeniden adlandır ve NaN'ları doldur
final_submission_df.columns = ['id', 'x', 'y']

# Kayıp satırları ortalama ile doldur (Format hatasını engeller)
final_submission_df['x'].fillna(final_submission_df['x'].mean(), inplace=True)
final_submission_df['y'].fillna(final_submission_df['y'].mean(), inplace=True)

# Final dosyayı kaydet
final_submission_df.to_csv('submission.csv', index=False)